In [1]:
import pandas as pd
import numpy as np

**Una Serie con Índices Múltiples**

* Comencemos considerando cómo podríamos representar datos bidimensionales dentro de una Serie unidimensional. Para ser concretos, consideraremos una serie de datos donde cada punto tiene una clave característica y numérica.

**La manera incorrecta**
* Supongamos que deseas seguir datos sobre estados de dos años diferentes. Usando las herramientas de Pandas que ya hemos cubierto, podrías sentir la tentación de simplemente usar tuplas de Python como claves:

In [24]:
index = [('California', 2000), ('California', 2010),
         ('New York', 2000), ('New York', 2010),
         ('Texas', 2000), ('Texas', 2010)
         ]

populations = [33871648, 37253956,
               18976457, 19378102,
               20851820, 25145561
               ]

pop = pd.Series(populations,index=index)
pop

(California, 2000)    33871648
(California, 2010)    37253956
(New York, 2000)      18976457
(New York, 2010)      19378102
(Texas, 2000)         20851820
(Texas, 2010)         25145561
dtype: int64

**Ejercicio Propio**

In [51]:
from itertools import groupby
from operator import itemgetter

# Paso 1: Definir los datos
index = [('California', 2000), ('California', 2010),
         ('New York', 2000), ('New York', 2010),
         ('Texas', 2000), ('Texas', 2010)
         ]

populations = [33871648, 37253956,
               18976457, 19378102,
               20851820, 25145561
               ]

# Paso 2: Crear una lista de tuplas (estado, año, población)
data = list(zip(index, populations))

# Paso 3: Ordenar la lista de tuplas por año usando lambda
data_sorted = sorted(data,key=lambda x: x[0][1])

# grouped_data = groupby(data_sorted,key=lambda x: x[0][1])
grouped_data = {key: list (group) for key, group in groupby(data_sorted,key=lambda x: x[0][1])}

for year, group in grouped_data.items():
    print(f"Año: {year}")
    for (state,_),population in group:
        print(f"  Estado: {state}, Población: {population}")        

Año: 2000
  Estado: California, Población: 33871648
  Estado: New York, Población: 18976457
  Estado: Texas, Población: 20851820
Año: 2010
  Estado: California, Población: 37253956
  Estado: New York, Población: 19378102
  Estado: Texas, Población: 25145561


Con este esquema de indexación, puedes indexar o dividir directamente la serie basándote en este índice múltiple:

In [3]:
pop[('California', 2010):('Texas', 2000)]

(California, 2010)    37253956
(New York, 2000)      18976457
(New York, 2010)      19378102
(Texas, 2000)         20851820
dtype: int64

pag 147